In [1]:
using Memoize
using DelimitedFiles
input = readlines("input.txt")
input[1] = replace(input[1], "Hit Points" => "HP") #doesnt compile without this, very important
hp = 50
mp = 500
input = [split(line, r":") for line in input]
input = Dict([(i[1], parse(Int, i[2])) for i in input])
input

Dict{SubString{String}, Int64} with 2 entries:
  "HP"     => 55
  "Damage" => 8

In [2]:
spells = readdlm("spells.csv", ';')
spells = convert(Array{Int}, spells[2:end, 2:end])

5×9 Matrix{Int64}:
  53  4  0  0  0  0  0    0  0
  73  2  2  0  0  0  0    0  0
 113  0  0  0  0  7  6    0  0
 173  0  0  3  6  0  0    0  0
 229  0  0  0  0  0  0  101  5

In [10]:
function part_one(;spells, boss_hp, boss_dmg, player_hp, player_mana, debug=false)
    min = Inf 
    #returns min mana needed to kill from a specific gamestate, cached
    
    @memoize function do_boss_turn(;bhp, php, pmp, bdot, bdot_dur, pa, pa_dur, pmreg, pmreg_dur, depth=0)
        if php <= 0
            return Inf, []
        end

        #tick dot effect
        if bdot_dur > 0
            bhp -= bdot
        end
        #check if boss is dead, if so it took 0 more mana
        if bhp <= 0
            return 0, []
        end
        #tick rest of effects
        bdot_dur -= 1
        pa_dur -= 1

        if pmreg_dur > 0
            pmp += pmreg
        end
        pmreg_dur -= 1
        #remove effects from expired effects
        if pa_dur <= 0 #since player armor doesn't matter at turn start, pa_dur does nothing benefitial at 1
            pa = 0
        end
        if pmreg_dur <= 0
            pmreg = 0
        end
        if bdot_dur <= 0
            bdot = 0
        end
        php = php - max(1, boss_dmg - pa)
        return sim_combat(bhp=bhp, php=php, pmp=pmp, bdot=bdot, 
            bdot_dur=bdot_dur, pa=pa, pa_dur=pa_dur, pmreg=pmreg, pmreg_dur=pmreg_dur, depth=depth)
    end
    
    @memoize function sim_combat(;bhp, php, pmp, bdot, bdot_dur, pa, pa_dur, pmreg, pmreg_dur, depth=0)
        #check if player is dead, if so it took Inf mana
        if php <= 0
            return Inf, []
        end

        #tick dot effect
        if bdot_dur > 0
            bhp -= bdot
        end
        #check if boss is dead, if so it took 0 more mana
        if bhp <= 0
            return 0, []
        end
        #tick rest of effects
        bdot_dur -= 1
        pa_dur -= 1

        if pmreg_dur > 0
            pmp += pmreg
        end
        pmreg_dur -= 1
        #remove effects from expired effects
        if pa_dur <= 0 #since player armor doesn't matter at turn start, pa_dur does nothing benefitial at 1
            pa = 0
        end
        if pmreg_dur <= 0
            pmreg = 0
        end
        if bdot_dur <= 0
            bdot = 0
        end
        min_from_here = Inf
        bpfh = []
        #cycle through usable spells
        for (ind, spell) in enumerate(eachrow(spells))
            _bdot_dur = bdot_dur
            _bdot = bdot
            _pa_dur = pa_dur
            _pa = pa
            _pmreg_dur = pmreg_dur
            _pmreg = pmreg
            #check if enough mp to cast
            if spell[1] > pmp
                continue
            end
            #check if its not an active effect
            if spell[5] > 0 #DoT
                if bdot_dur > 0
                    continue
                end
                _bdot_dur = spell[5]
                _bdot = spell[4]
            end
            if spell[7] > 0 #player armor
                if pa_dur > 0
                    continue
                end
                _pa_dur = spell[7]
                _pa = spell[6]
            end
            if spell[9] > 0
                if pmreg_dur > 0
                    continue
                end
                _pmreg_dur = spell[9]
                _pmreg = spell[8]
            end
            if debug
                tabs = reduce(*, ["-" for x in 1:depth])
                print("$tabs Casting $ind at state:")
                println(" $bhp|$php|$pmp|$bdot|$bdot_dur|$pa|$pa_dur|$pmreg|$pmreg_dur")
            end
            min_from_spell, best_path = do_boss_turn(bhp=bhp-spell[2], php=php+spell[3], pmp=pmp-spell[1], bdot=_bdot, 
            bdot_dur=_bdot_dur, pa=_pa, pa_dur=_pa_dur, pmreg=_pmreg, pmreg_dur=_pmreg_dur, depth=depth+1)
            min_from_spell = spell[1] + min_from_spell
            if debug
                println("$tabs Outcome: $min_from_spell with path: $best_path")
            end
            best_path = vcat([ind], best_path)
            if min_from_here >= min_from_spell
                min_from_here = min_from_spell
                bpfh = best_path
            end
        end
        return min_from_here, bpfh
    end
    return sim_combat(bhp = boss_hp, php = player_hp, pmp = player_mana, bdot = 0, 
                       bdot_dur = 0, pa = 0, pa_dur = 0, pmreg = 0, pmreg_dur = 0)
end

part_one(spells = spells, boss_hp = input["HP"], boss_dmg = input["Damage"], player_hp = hp, player_mana = mp)

(953, Any[5, 4, 3, 1, 4, 1, 1, 1, 1])

In [9]:
function part_two(;spells, boss_hp, boss_dmg, player_hp, player_mana, debug=false)
    min = Inf 
    #returns min mana needed to kill from a specific gamestate, cached
    
    @memoize function do_boss_turn(;bhp, php, pmp, bdot, bdot_dur, pa, pa_dur, pmreg, pmreg_dur, depth=0)
        if php <= 0
            return Inf, []
        end

        #tick dot effect
        if bdot_dur > 0
            bhp -= bdot
        end
        #check if boss is dead, if so it took 0 more mana
        if bhp <= 0
            return 0, []
        end
        #tick rest of effects
        bdot_dur -= 1
        pa_dur -= 1

        if pmreg_dur > 0
            pmp += pmreg
        end
        pmreg_dur -= 1
        #remove effects from expired effects
        if pa_dur <= 0 #since player armor doesn't matter at turn start, pa_dur does nothing benefitial at 1
            pa = 0
        end
        if pmreg_dur <= 0
            pmreg = 0
        end
        if bdot_dur <= 0
            bdot = 0
        end
        php = php - max(1, boss_dmg - pa)
        return sim_combat(bhp=bhp, php=php, pmp=pmp, bdot=bdot, 
            bdot_dur=bdot_dur, pa=pa, pa_dur=pa_dur, pmreg=pmreg, pmreg_dur=pmreg_dur, depth=depth)
    end
    
    @memoize function sim_combat(;bhp, php, pmp, bdot, bdot_dur, pa, pa_dur, pmreg, pmreg_dur, depth=0)
        #check if player is dead, if so it took Inf mana
        php -= 1
        if php <= 0
            return Inf, []
        end

        #tick dot effect
        if bdot_dur > 0
            bhp -= bdot
        end
        #check if boss is dead, if so it took 0 more mana
        if bhp <= 0
            return 0, []
        end
        #tick rest of effects
        bdot_dur -= 1
        pa_dur -= 1

        if pmreg_dur > 0
            pmp += pmreg
        end
        pmreg_dur -= 1
        #remove effects from expired effects
        if pa_dur <= 0 #since player armor doesn't matter at turn start, pa_dur does nothing benefitial at 1
            pa = 0
        end
        if pmreg_dur <= 0
            pmreg = 0
        end
        if bdot_dur <= 0
            bdot = 0
        end
        min_from_here = Inf
        bpfh = []
        #cycle through usable spells
        for (ind, spell) in enumerate(eachrow(spells))
            _bdot_dur = bdot_dur
            _bdot = bdot
            _pa_dur = pa_dur
            _pa = pa
            _pmreg_dur = pmreg_dur
            _pmreg = pmreg
            #check if enough mp to cast
            if spell[1] > pmp
                continue
            end
            #check if its not an active effect
            if spell[5] > 0 #DoT
                if bdot_dur > 0
                    continue
                end
                _bdot_dur = spell[5]
                _bdot = spell[4]
            end
            if spell[7] > 0 #player armor
                if pa_dur > 0
                    continue
                end
                _pa_dur = spell[7]
                _pa = spell[6]
            end
            if spell[9] > 0
                if pmreg_dur > 0
                    continue
                end
                _pmreg_dur = spell[9]
                _pmreg = spell[8]
            end
            if debug
                tabs = reduce(*, ["-" for x in 1:depth])
                print("$tabs Casting $ind at state:")
                println(" $bhp|$php|$pmp|$bdot|$bdot_dur|$pa|$pa_dur|$pmreg|$pmreg_dur")
            end
            min_from_spell, best_path = do_boss_turn(bhp=bhp-spell[2], php=php+spell[3], pmp=pmp-spell[1], bdot=_bdot, 
            bdot_dur=_bdot_dur, pa=_pa, pa_dur=_pa_dur, pmreg=_pmreg, pmreg_dur=_pmreg_dur, depth=depth+1)
            min_from_spell = spell[1] + min_from_spell
            if debug
                println("$tabs Outcome: $min_from_spell with path: $best_path")
            end
            best_path = vcat([ind], best_path)
            if min_from_here >= min_from_spell
                min_from_here = min_from_spell
                bpfh = best_path
            end
        end
        return min_from_here, bpfh
    end
    return sim_combat(bhp = boss_hp, php = player_hp, pmp = player_mana, bdot = 0, 
                       bdot_dur = 0, pa = 0, pa_dur = 0, pmreg = 0, pmreg_dur = 0)
end

part_two(spells = spells, boss_hp = input["HP"], boss_dmg = input["Damage"], player_hp = hp, player_mana = mp)

(1289, Any[4, 5, 3, 4, 5, 2, 4, 2, 1])

In [4]:
mm = spells[1, :]
drain = spells[2, :]
shield = spells[3, :]
poison = spells[4, :]
rech = spells[5, :]
blank = zeros(Int, 9)
spells

5×9 Matrix{Int64}:
  53  4  0  0  0  0  0    0  0
  73  2  2  0  0  0  0    0  0
 113  0  0  0  0  7  6    0  0
 173  0  0  3  6  0  0    0  0
 229  0  0  0  0  0  0  101  5

In [5]:
tstspells = hcat(shield, poison, blank)'
part_one(spells = tstspells, boss_hp = 18, boss_dmg = 8, player_hp = 10, player_mana = 1000, debug=true)

 Casting 1 at state: 18|10|1000|0|-1|0|-1|0|-1
- Casting 2 at state: 18|9|887|0|-3|7|4|0|-3
-- Casting 3 at state: 12|8|714|3|4|7|2|0|-5
--- Casting 1 at state: 6|7|714|3|2|0|0|0|-7
--- Outcome: 113 with path: Any[]
--- Casting 3 at state: 6|7|714|3|2|0|0|0|-7
--- Outcome: Inf with path: Any[]
-- Outcome: 113 with path: Any[1]
- Outcome: 286 with path: Any[3, 1]
- Casting 3 at state: 18|9|887|0|-3|7|4|0|-3
-- Casting 2 at state: 18|8|887|0|-5|7|2|0|-5
--- Casting 1 at state: 12|7|714|3|4|0|0|0|-7
---- Casting 3 at state: 6|6|601|3|2|7|4|0|-9
---- Outcome: 0 with path: Any[]
--- Outcome: 113 with path: Any[3]
--- Casting 3 at state: 12|7|714|3|4|0|0|0|-7
--- Outcome: Inf with path: Any[]
-- Outcome: 286 with path: Any[1, 3]
-- Casting 3 at state: 18|8|887|0|-5|7|2|0|-5
--- Casting 1 at state: 18|7|887|0|-7|0|0|0|-7
---- Casting 2 at state: 18|6|774|0|-9|7|4|0|-9
----- Casting 3 at state: 12|5|601|3|4|7|2|0|-11
------ Casting 1 at state: 6|4|601|3|2|0|0|0|-13
------ Outcome: 113 with pat

(399, Any[1, 3, 2, 1, 3])